In [1]:
import math
import numpy as np
import sys
import tensorflow as tf

sys.path.append('/home/dong/Dropbox/Projects/NLP/seq2seq')
from seq2seq.encoders import rnn_encoder
from seq2seq.decoders import basic_decoder

In [2]:
from seq2seq.contrib.seq2seq import decoder as contrib_decoder

In [3]:
PAD = 0
EOS = 1


vocab_size = 10
input_embedding_size = 50

# 第一层的encoder RNN cell 的 hidden_state_size
encoder1_hidden_units = 50
# 第二层的encoder RNN cell 的 hidden_state_size
# 因为要记忆相对大量的context，所以 “*2”
encoder2_hidden_units = encoder1_hidden_units * 2

# decoder 的 hidden_state_size
# encoder2 使用 unidirectional_rnn
# 注意 encoder2 不能使用 bidirectional_rnn
decoder_hidden_units = encoder2_hidden_units

import helpers as data_helpers
batch_size = 11
round_num = 20

# 一个generator，每次产生一个minibatch的随机样本

batches = data_helpers.random_sequences(length_from=3, length_to=8,
                                   vocab_lower=2, vocab_upper=10,
                                   batch_size=batch_size*round_num)

### 打印demo数据

In [4]:
def demo_mult_rounds(batches, batch_size, round_num):
    data = next(batches)
    mb = list()
    id = 0
    for i in range(batch_size):
        mb.append([])
        for j in range(round_num):
            mb[-1].append(data[id])
            id += 1
    return mb

print('产生%d组的sequences, \n'
      '每一组sequence包含%d句长度不一（最短3，最长8）的sequence, \n'
      '其中前十组是:\n' % (batch_size, round_num))

for seq in demo_mult_rounds(batches, batch_size, round_num):
    print('%s\n' % seq)

产生11组的sequences, 
每一组sequence包含20句长度不一（最短3，最长8）的sequence, 
其中前十组是:

[[6, 7, 9, 7, 9, 2, 3], [6, 3, 8, 5, 5], [3, 6, 2, 4, 3, 2, 2, 8], [6, 5, 3, 9, 9, 7], [3, 5, 6], [9, 5, 2, 8, 5], [3, 6, 5, 9, 5, 3, 2, 4], [8, 7, 8, 7], [8, 3, 8, 7], [3, 6, 3, 5, 8, 5, 7], [3, 3, 2, 3, 6, 6], [7, 9, 5, 7, 3, 6], [9, 9, 7, 3, 2], [8, 7, 2, 2, 7, 7, 4], [8, 2, 9], [8, 6, 6, 6], [7, 4, 4, 6, 4], [2, 9, 6, 7, 8], [2, 7, 3, 6, 7, 9, 9], [2, 2, 4]]

[[8, 6, 5, 2, 5, 9, 7, 5], [8, 6, 4], [5, 3, 9, 2, 6, 9, 9], [8, 2, 6, 7, 8], [5, 8, 3], [3, 7, 3, 5, 5, 8], [3, 8, 7, 4, 6, 6, 4, 2], [6, 4, 4, 2, 8, 7, 3, 3], [9, 5, 6, 3, 9, 9, 8], [2, 7, 3, 5, 8, 5], [4, 3, 7, 5, 5, 3, 2], [6, 6, 4, 9, 2, 6, 4, 3], [8, 3, 5, 7, 7, 7], [8, 9, 3, 4, 2, 2, 8], [4, 5, 3, 5, 8], [3, 9, 6, 5], [8, 5, 9], [7, 3, 2, 7, 8, 6], [3, 7, 7, 5, 7], [6, 8, 6, 8, 5, 5]]

[[3, 6, 6], [3, 3, 9, 9, 7, 8, 9], [9, 2, 3, 7, 5, 3, 9], [6, 6, 2, 3, 2, 7, 5], [3, 9, 2], [7, 7, 6, 7, 2, 5, 4], [7, 6, 8, 8, 4, 5, 3], [3, 6, 9], [3, 9, 2, 6, 9, 2, 2]

## 产生轮数为20的合成数据

#### 使用连续20个sequence模拟一个轮数为20的对话数据

#### 第i-轮的decoder输出是从第1句到第i-句输入的拼接

In [5]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
mode = tf.contrib.learn.ModeKeys.TRAIN

In [6]:
with tf.name_scope('minibatch_encoder1'):
    # 一个 minibatch 包含 batch_size * round_num 个 sequences
    encoder1_inputs = tf.placeholder(shape=(batch_size*round_num, None),
                                     dtype=tf.int32,
                                     name='encoder1_inputs')
    encoder1_inputs_length = tf.placeholder(shape=(batch_size*round_num,),
                                            dtype=tf.int32,
                                            name='encoder1_inputs_length')

with tf.name_scope('minibatch_encoder2'):
    encoder2_inputs_length = tf.placeholder(shape=(batch_size,),
                                            dtype=tf.int32,
                                            name='encoder2_inputs_length')

with tf.name_scope('minibatch_decoder'):    
    decoder_targets = tf.placeholder(shape=(batch_size*round_num, None),
                                     dtype=tf.int32,
                                     name='decoder_targets')
    
    decoder_inputs = tf.placeholder(shape=(batch_size*round_num, None),
                                    dtype=tf.int32,
                                    name='decoder_inputs')
    decoder_inputs_length = tf.placeholder(shape=(batch_size*round_num,),
                                            dtype=tf.int32,
                                            name='decoder_inputs_length')

## encoding阶段

### 第一层encoder
* `encoder1_output` 的 `final_state` 用做 decoder 的 `initial_state`
* `encoder1_output` 的 `final_state` 也用做第二层 encoder 的 `input`

In [7]:
# 每个句子encoding的超参数
encoder1_params = rnn_encoder.StackBidirectionalRNNEncoder.default_params()
encoder1_params["rnn_cell"]["cell_params"]["num_units"] = encoder1_hidden_units
encoder1_params["rnn_cell"]["cell_class"] = "BasicLSTMCell"
encoder1_params["rnn_cell"]["num_layers"] = 2
encoder1_params

{'init_scale': 0.04,
 'rnn_cell': {'cell_class': 'BasicLSTMCell',
  'cell_params': {'num_units': 50},
  'dropout_input_keep_prob': 1.0,
  'dropout_output_keep_prob': 1.0,
  'num_layers': 2,
  'residual_combiner': 'add',
  'residual_connections': False,
  'residual_dense': False}}

In [8]:
# 第一层 embedding
with tf.name_scope('embedding'):
    input_embeddings = tf.Variable(
        tf.random_uniform([vocab_size, input_embedding_size], -1.0, 1.0),
        dtype=tf.float32)

with tf.name_scope('ENC-level1'):

    encoder1_inputs_embedded = tf.nn.embedding_lookup(
        input_embeddings, encoder1_inputs)
    encode_fn1 = rnn_encoder.StackBidirectionalRNNEncoder(
        encoder1_params, mode)
    encoder1_output = encode_fn1(
        encoder1_inputs_embedded, encoder1_inputs_length)

INFO:tensorflow:Creating StackBidirectionalRNNEncoder in mode=train
INFO:tensorflow:
StackBidirectionalRNNEncoder:
  init_scale: 0.04
  rnn_cell:
    cell_class: BasicLSTMCell
    cell_params: {num_units: 50}
    dropout_input_keep_prob: 1.0
    dropout_output_keep_prob: 1.0
    num_layers: 2
    residual_combiner: add
    residual_connections: false
    residual_dense: false



In [9]:
print('outputs: %s\n\n' % repr(encoder1_output.outputs))
print('final state: %s\n\n' % repr(encoder1_output.final_state))

outputs: <tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/concat:0' shape=(220, ?, 100) dtype=float32>


final state: ((LSTMStateTuple(c=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 50) dtype=float32>, h=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 50) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 50) dtype=float32>, h=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 50) dtype=float32>)), (LSTMStateTuple(c=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 50) dtype=float32>, h=<tf.Tensor 'ENC-level1/stacked_bidi_rnn_enco

##### `google/seq2seq`使用`tf.contrib.rnn.stack_bidirectional_dynamic_rnn`, 不包含time-major选项，使用batch-major inputs 数据；`tf.nn.bidirectional_dynamic_rnn`包含time-major选项

```python
result = rnn.stack_bidirectional_dynamic_rnn(
    cells_fw=cells_fw,
    cells_bw=cells_bw,
    inputs=inputs,
    dtype=tf.float32,
    sequence_length=sequence_length,
    **kwargs)
outputs_concat, _output_state_fw, _output_state_bw = result
final_state = (_output_state_fw, _output_state_bw)
```

case study:

**outputs:** 自动concatenate以后的`(batch_size, seq_length, 2*encoder1_hidden_units)`的tensor
```
<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/concat:0' shape=(?, ?, 50) dtype=float32>
```

**final state：** 两层，双向的LSTM-RNN，共有四个LSTMStateTuple
```
final state: (
  (
    LSTMStateTuple(
      c=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 25) dtype=float32>, 
      h=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 25) dtype=float32> ),   
    LSTMStateTuple(
      c=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 25) dtype=float32>, 
      h=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 25) dtype=float32> )
  ),
  (
    LSTMStateTuple(
      c=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 25) dtype=float32>, 
      h=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_0/bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 25) dtype=float32>),     
    LSTMStateTuple(
      c=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/bw/while/Exit_2:0' shape=(?, 25) dtype=float32>, 
      h=<tf.Tensor 'stacked_bidi_rnn_encoder/stack_bidirectional_rnn/cell_1/bidirectional_rnn/bw/bw/while/Exit_3:0' shape=(?, 25) dtype=float32>)
  )
)
```

#### 处理第一层encoder的输出

In [10]:
with tf.name_scope('level1-states'):
    encoder1_final_state_c = tf.concat(
        (encoder1_output.final_state[0][1].c, 
         encoder1_output.final_state[1][1].c), 
        1)

    encoder1_final_state_h = tf.concat(
        (encoder1_output.final_state[0][1].h,
         encoder1_output.final_state[1][1].h),
        1)

    encoder1_final_state = tf.nn.rnn_cell.LSTMStateTuple(
        c=encoder1_final_state_c,
        h=encoder1_final_state_h
    )

In [11]:
encoder1_final_state

LSTMStateTuple(c=<tf.Tensor 'level1-states/concat:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'level1-states/concat_1:0' shape=(?, 100) dtype=float32>)

### 第二层encoder


In [12]:
# 将context做encoding的超参数
encoder2_params = rnn_encoder.UnidirectionalRNNEncoder.default_params()
encoder2_params["rnn_cell"]["cell_params"]["num_units"] = encoder2_hidden_units
encoder2_params["rnn_cell"]["cell_class"] = "BasicLSTMCell"
encoder2_params["rnn_cell"]["num_layers"] = 2
encoder2_params

{'init_scale': 0.04,
 'rnn_cell': {'cell_class': 'BasicLSTMCell',
  'cell_params': {'num_units': 100},
  'dropout_input_keep_prob': 1.0,
  'dropout_output_keep_prob': 1.0,
  'num_layers': 2,
  'residual_combiner': 'add',
  'residual_connections': False,
  'residual_dense': False}}

In [13]:
# 第二层 embedding
print(repr(encoder1_final_state))

with tf.name_scope('ENC-level2'):
    # 1. reshape from (batch_size x round_num, hidden_state_size)
    #   to (batch_size, round_num, hidden_state_size)
    encoder2_inputs = tf.reshape(encoder1_final_state.h,
                                 [-1, round_num, encoder2_hidden_units])

    # 2. 共batch_size个样本，每个样本长度为 round_num, 每个元素是一个原始样本的rnn_encoder_final_state
    encode_fn2 = rnn_encoder.UnidirectionalRNNEncoder(
        encoder2_params, mode)
    encoder2_output = encode_fn2(encoder2_inputs, encoder2_inputs_length)

LSTMStateTuple(c=<tf.Tensor 'level1-states/concat:0' shape=(?, 100) dtype=float32>, h=<tf.Tensor 'level1-states/concat_1:0' shape=(?, 100) dtype=float32>)
INFO:tensorflow:Creating UnidirectionalRNNEncoder in mode=train
INFO:tensorflow:
UnidirectionalRNNEncoder:
  init_scale: 0.04
  rnn_cell:
    cell_class: BasicLSTMCell
    cell_params: {num_units: 100}
    dropout_input_keep_prob: 1.0
    dropout_output_keep_prob: 1.0
    num_layers: 2
    residual_combiner: add
    residual_connections: false
    residual_dense: false



In [14]:
# 3. 将batch_size个样本的各个round_num个元素的 output 作为 decoding context
with tf.name_scope('level2_outputs'):
    context_state = tf.reshape(
        encoder2_output.outputs,
        [batch_size * round_num, encoder2_hidden_units])

## decoding阶段

In [15]:
# 准备新的输入

with tf.name_scope('decoder_input'):
    decoder_inputs_embedded = tf.nn.embedding_lookup(
        input_embeddings, decoder_inputs)
    
    context_state = tf.tile(context_state,
                            [1, tf.shape(decoder_inputs_embedded)[1]])
    context_state = tf.reshape(context_state, 
                               [batch_size*round_num,
                                tf.shape(decoder_inputs_embedded)[1],
                                encoder2_hidden_units])

    decoder_inputs_embedded = tf.concat(
        (decoder_inputs_embedded,
         context_state
        ), axis = -1
    )


In [16]:
from seq2seq.contrib.seq2seq import helper as decode_helper
with tf.name_scope('decoder_helper'):
    helper_ = decode_helper.TrainingHelper(
        inputs = decoder_inputs_embedded,
        sequence_length = decoder_inputs_length)

In [17]:
decode_params = basic_decoder.BasicDecoder.default_params()
decode_params["rnn_cell"]["cell_params"]["num_units"] = decoder_hidden_units
decode_params["max_decode_length"] = batch_size * round_num + 5
decode_params

{'init_scale': 0.04,
 'max_decode_length': 225,
 'rnn_cell': {'cell_class': 'BasicLSTMCell',
  'cell_params': {'num_units': 100},
  'dropout_input_keep_prob': 1.0,
  'dropout_output_keep_prob': 1.0,
  'num_layers': 1,
  'residual_combiner': 'add',
  'residual_connections': False,
  'residual_dense': False}}

In [18]:
with tf.name_scope('decoder'):
    decoder_fn = basic_decoder.BasicDecoder(params=decode_params,
                                            mode=mode,
                                            vocab_size=vocab_size)
    decoder_output, decoder_state = decoder_fn(
        encoder1_final_state,
        helper_)

INFO:tensorflow:Creating BasicDecoder in mode=train
INFO:tensorflow:
BasicDecoder:
  init_scale: 0.04
  max_decode_length: 225
  rnn_cell:
    cell_class: BasicLSTMCell
    cell_params: {num_units: 100}
    dropout_input_keep_prob: 1.0
    dropout_output_keep_prob: 1.0
    num_layers: 1
    residual_combiner: add
    residual_connections: false
    residual_dense: false



In [19]:
#print('只对每一组数据的最后一个时间，即所有sequence拼接以后的sequence，计算loss')
#[[x] for x in range(round_num-1, batch_size*round_num, round_num)]

In [20]:
with tf.name_scope('loss'):
    indices = tf.constant(
        [[x] for x in range(round_num-1, batch_size*round_num, round_num)],
        dtype=tf.int32)

    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(tf.gather_nd(params = decoder_targets,
                                           indices = indices),
                              depth=vocab_size, dtype=tf.float32),
            logits=tf.gather_nd(params = tf.transpose(decoder_output.logits,
                                             perm = [1, 0, 2]),
                               indices = indices)
        )
    )

    train_op = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

In [21]:
sess.run(tf.global_variables_initializer())

In [23]:
import os 

log_path = os.path.join(os.getcwd(), 'arch-hred-basic')
summary_writer = tf.summary.FileWriter(log_path, sess.graph)


# 保存模型
# word2vec参数的单词和词向量部分分别保存到了metadata和ckpt文件里面
saver = tf.train.Saver()
saver.save(sess, os.path.join(log_path, "model.ckpt"))

'/home/dong/Dropbox/Projects/NLP/lecture5/arch-hred-basic/model.ckpt'

## 训练阶段

In [ ]:
batch = next(batches)

cumbatch = []
for i in range(len(batch)):
    if i%round_num==0:
        cumbatch.append(batch[i])
    else:
        cumbatch.append(batch[i] + cumbatch[-1])

encoder_inputs_, encoder_inputs_length_ = data_helpers.batch(batch)
decoder_targets_, _ = data_helpers.batch(
    [(sequence) + [EOS] for sequence in cumbatch]
)
decoder_inputs_, decoder_inputs_length_ = data_helpers.batch(
    [[EOS] + (sequence) for sequence in cumbatch]
)

In [ ]:
decoder_targets_.T[:5,:]


In [ ]:
decoder_inputs_.T[:5,:]

In [ ]:
def next_feed():
    batch = next(batches)

    cumbatch = []
    for i in range(len(batch)):
        if i%round_num==0:
            cumbatch.append(batch[i])
        else:
            cumbatch.append(batch[i] + cumbatch[-1])

    encoder_inputs_, encoder1_inputs_length_ = data_helpers.batch(batch)
    encoder2_inputs_length_ = np.array([round_num]*batch_size)
    decoder_targets_, _ = data_helpers.batch(
        [(sequence) + [EOS] for sequence in cumbatch]
    )
    decoder_inputs_, decoder_inputs_length_ = data_helpers.batch(
        [[EOS] + (sequence) for sequence in cumbatch]
    )    
    # 在feedDict里面，key可以是一个Tensor
    return {
        encoder1_inputs: encoder_inputs_.T,
        decoder_inputs: decoder_inputs_.T,
        decoder_targets: decoder_targets_.T,
        encoder1_inputs_length: encoder1_inputs_length_,
        encoder2_inputs_length: encoder2_inputs_length_,
        decoder_inputs_length: decoder_inputs_length_
    }

In [ ]:
loss_track = []
max_batches = 3001
batches_in_epoch = 100

try:
    # 一个epoch的learning
    for batch in range(max_batches):
        fd = next_feed()
        _, l = sess.run([train_op, loss], fd)
        loss_track.append(l)
        
        if batch == 0 or batch % batches_in_epoch == 0:
            print('batch {}'.format(batch))
            print('  minibatch loss: {}'.format(sess.run(loss, fd)))
            predict_ = sess.run(decoder_output.predicted_ids, fd)
            for i, (inp, targ, pred) in enumerate(
                zip(fd[encoder1_inputs], 
                    fd[decoder_targets], 
                    predict_.T)):
                if i in [0, round_num-1]:
                    print('  sample {}:'.format(i + 1))
                    print('    targets     > {}'.format(targ))
                    print('    predicted > {}'.format(pred))
                if i == round_num-1:
                    break
            print()
        
except KeyboardInterrupt:
    print('training interrupted')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(loss_track)
print('loss {:.4f} after {} examples (batch_size={})'.format(loss_track[-1], len(loss_track)*batch_size, batch_size))